# Отток клиентов

Сфера деятельности - банковская.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Метрики качества - *F1-мера, AUC ROC.*

# 1. Подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('/datasets/Churn.csv')

In [3]:
# Изучим данные
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [4]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [6]:
# приведем все к нижнему регистру
data.columns = map(str.lower, data.columns)
data = data.rename(columns={'rownumber': 'row_number', 
                            'customerid': 'customer_id', 
                            'creditscore':'credit_score', 
                            'numofproducts': 'num_of_products', 
                            'hascrcard':'has_cr_card', 
                            'isactivemember':'is_active_member', 
                            'estimatedsalary':'estimated_salary'})
def lower_object_string(df):
    if df.dtype=='object':
        df = df.str.lower()
    return df
data = data.apply(lambda x: lower_object_string(x))
# проверим
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
row_number          10000 non-null int64
customer_id         10000 non-null int64
surname             10000 non-null object
credit_score        10000 non-null int64
geography           10000 non-null object
gender              10000 non-null object
age                 10000 non-null int64
tenure              9091 non-null float64
balance             10000 non-null float64
num_of_products     10000 non-null int64
has_cr_card         10000 non-null int64
is_active_member    10000 non-null int64
estimated_salary    10000 non-null float64
exited              10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [7]:
data['surname'].value_counts()

smith      32
martin     29
scott      29
walker     28
brown      26
           ..
wells       1
zarate      1
mauldon     1
stanton     1
lindon      1
Name: surname, Length: 2931, dtype: int64

In [8]:
data['gender'].value_counts()

male      5457
female    4543
Name: gender, dtype: int64

In [9]:
data['geography'].value_counts()

france     5014
germany    2509
spain      2477
Name: geography, dtype: int64

In [10]:
# Посмотрим, есть ли какая-нибудь взаимосвязь столбца с пропущеннными данными (количество недвижимости) по сравнению с другими
data.groupby('tenure')['balance'].mean()
# Явной зависимости нет

tenure
0.0     80135.307277
1.0     77365.221712
2.0     76502.413053
3.0     78105.560356
4.0     74133.600373
5.0     76138.007217
6.0     74970.085698
7.0     75915.378086
8.0     77994.797685
9.0     77325.410351
10.0    73580.495964
Name: balance, dtype: float64

In [11]:
data.groupby('tenure')['age'].mean()
# тоже самое, явной зависимости не наблюдается

tenure
0.0     39.306283
1.0     39.646008
2.0     39.246316
3.0     38.984914
4.0     38.116384
5.0     39.242718
6.0     38.637911
7.0     38.349189
8.0     38.496249
9.0     39.506803
10.0    39.195067
Name: age, dtype: float64

In [12]:
data.groupby('tenure')['estimated_salary'].mean()
# аналогичная ситуация

tenure
0.0      99565.395157
1.0     100635.368718
2.0      99120.730168
3.0      97351.648168
4.0     100041.330836
5.0     101239.472233
6.0     100151.566924
7.0     101842.455362
8.0     100611.163290
9.0      99592.441995
10.0    102841.772040
Name: estimated_salary, dtype: float64

In [13]:
data.groupby('tenure')['credit_score'].mean()
# нет явной зависимости

tenure
0.0     651.913613
1.0     648.681723
2.0     654.644211
3.0     651.096983
4.0     650.681356
5.0     649.962244
6.0     647.757094
7.0     649.195676
8.0     648.618435
9.0     655.086168
10.0    651.670404
Name: credit_score, dtype: float64

In [14]:
# Поэтому наилучшим вариантом будет заменить пропущенные данные медианами
data['tenure'] = data['tenure'].fillna(data['tenure'].median())

In [15]:
# Поменяем тип данных для balance, estimated_salary и tenure, так как удобнее будет работать с целочисленными данными
data['balance'] = data['balance'].astype('int')
data['estimated_salary'] = data['estimated_salary'].astype('int')
data['tenure'] = data['tenure'].astype('int')

In [16]:
# Применим прямое кодирование
data_ohe = data.drop(['row_number', 'customer_id', 'surname'], axis=1)
data_ohe = pd.get_dummies(data_ohe, drop_first=True)
data_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
credit_score         10000 non-null int64
age                  10000 non-null int64
tenure               10000 non-null int64
balance              10000 non-null int64
num_of_products      10000 non-null int64
has_cr_card          10000 non-null int64
is_active_member     10000 non-null int64
estimated_salary     10000 non-null int64
exited               10000 non-null int64
geography_germany    10000 non-null uint8
geography_spain      10000 non-null uint8
gender_male          10000 non-null uint8
dtypes: int64(9), uint8(3)
memory usage: 732.5 KB


In [17]:
# Разобьем исходные данные на обучающую (df_train), валидационную (df_val) и тестовую выборки (df_test) в соотношении 3:1:1
data_train, data_test = train_test_split(data_ohe, test_size=0.2, random_state=12345)
data_train, data_valid = train_test_split(data_train, test_size=0.25, random_state=12345)
# Проверим получилось ли разделить данные в необходимом соотношении
data_train.shape[0], data_test.shape[0], data_valid.shape[0]

(6000, 2000, 2000)

In [18]:
features_train = data_train.drop(['exited'], axis=1)
target_train = data_train['exited']
features_valid = data_valid.drop(['exited'], axis=1)
target_valid = data_valid['exited']
features_test = data_test.drop(['exited'], axis=1)
target_test = data_test['exited']
features_test.shape, features_valid.shape, features_train.shape

((2000, 11), (2000, 11), (6000, 11))

In [19]:
# Масштабируем признаки      
numeric = ['credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'estimated_salary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])



### Вывод

На данном этапе данные были приведены к нижнему регистру как индексы, так и названия колонок, были заменены пропуски в столбце tenure на заглушки, поменяли типы данных для столбцов balance, estimated_salary (нет необходимости видеть суммы до копеек) и tenure (количество недвижимости не может быть нецелочисленным значением).

# 2. Исследование задачи

Рассмотрим первую модель - решающее дерево.

In [20]:
dt = DecisionTreeClassifier(random_state = 12345)
parameters_dt = {'criterion': ['gini', 'entropy'], # критерий
                 'max_depth':range(1, 50), # глубина дерева
                 'min_samples_leaf': range(1, 20), # минимальное количесто объектов в листе
                 'max_features':range(1, features_train.shape[1]+1)} # максимальное количество фич
search_dt = RandomizedSearchCV(dt, parameters_dt, cv=5, n_jobs = -1, scoring = 'f1')
search_dt.fit(features_train, target_train)
best_dt = search_dt.best_estimator_
f1_score_val_dt = cross_val_score(best_dt, features_valid, target_valid, scoring = 'f1', cv = 10, n_jobs = -1)   
auc_roc_val_dt = cross_val_score(best_dt, features_valid, target_valid, scoring = 'roc_auc', cv = 10, n_jobs = -1)
print('F1-score для Decision Tree на валидационной выборке:', sum(f1_score_val_dt)/len(f1_score_val_dt))
print('AUC-ROC для Decision Tree на валидационной выборке:', sum(auc_roc_val_dt)/len(auc_roc_val_dt))

F1_score для Decision Tree на валидационной выборке: 0.5096693267584176
AUC-ROC для Decision Tree на валидационной выборке: 0.7580816013696449


Вторая модель - случайный лес.

In [21]:
rf = RandomForestClassifier(random_state = 12345)
parameters_rf = {'n_estimators': range(1, 200, 10), # количество деревьев 
                 'criterion': ['gini', 'entropy'], # критерий
                 'max_depth':range(1, 50, 1), # глубина дерева
                 'min_samples_leaf': range(1, 10), # минимальное количесто объектов в листе
                 'max_features': range(1, features_train.shape[1]+1)} # максимальное количество фич
search_rf = RandomizedSearchCV(rf, parameters_rf, cv=5, n_jobs = -1, scoring = 'f1')
search_rf.fit(features_train, target_train)
best_rf = search_rf.best_estimator_
f1_score_val_rf = cross_val_score(best_rf, features_valid, target_valid, scoring = 'f1', cv = 10, n_jobs = -1)   
auc_roc_val_rf = cross_val_score(best_rf, features_valid, target_valid, scoring = 'roc_auc', cv = 10, n_jobs = -1)
print('F1-score для Random Forest на валидационной выборке:', sum(f1_score_val_rf)/len(f1_score_val_rf))
print('AUC-ROC для Random Forest на валидационной выборке:', sum(auc_roc_val_rf)/len(auc_roc_val_rf))

F1_score для Random Forest на валидационной выборке: 0.5316937017557966
AUC-ROC для Random Forest на валидационной выборке: 0.8293713170887085


Третья модель - логистическая регрессия.

In [22]:
lr = LogisticRegression(random_state = 12345)
parameters_lr_1 = {'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 5, 10, 20, 50], # обратная сила регуляризации
                 'penalty':['l1'], # Регуляризация 
                 'solver':['liblinear', 'saga']} # функция потерь
parameters_lr_2 = {'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 5, 10, 20, 50], # обратная сила регуляризации
                 'penalty':['l2'], # Регуляризация 
                 'solver':['lbfgs', 'saga', 'sag', 'newton-cg']} # функция потерь

search_lr_1 = RandomizedSearchCV(lr, parameters_lr_1, cv=5, n_jobs = -1, scoring = 'f1')
search_lr_1.fit(features_train, target_train)
best_lr_1 = search_lr_1.best_estimator_
f1_score_val_lr_1 = cross_val_score(best_lr_1, features_valid, target_valid, scoring = 'f1', cv = 10, n_jobs = -1)   
auc_roc_val_lr_1 = cross_val_score(best_lr_1, features_valid, target_valid, scoring = 'roc_auc', cv = 10, n_jobs = -1) 


search_lr_2 = RandomizedSearchCV(lr, parameters_lr_2, cv=5, n_jobs = -1, scoring = 'f1')
search_lr_2.fit(features_train, target_train)
best_lr_2 = search_lr_2.best_estimator_
f1_score_val_lr_2 = cross_val_score(best_lr_2, features_valid, target_valid, scoring = 'f1', cv = 10, n_jobs = -1)   
auc_roc_val_lr_2 = cross_val_score(best_lr_2, features_valid, target_valid, scoring = 'roc_auc', cv = 10, n_jobs = -1) 
print('F1-score для Logistic Regression (l1) на валидационной выборке:', sum(f1_score_val_lr_1)/len(f1_score_val_lr_1))
print('AUC-ROC для Logistic Regression (l1) на валидационной выборке:', sum(auc_roc_val_lr_1)/len(auc_roc_val_lr_1))
print()
print('F1-score для Logistic Regression (l2) на валидационной выборке:', sum(f1_score_val_lr_2)/len(f1_score_val_lr_2))
print('AUC-ROC для Logistic Regression (l2) на валидационной выборке:', sum(auc_roc_val_lr_2)/len(auc_roc_val_lr_2))

F1_score для Logistic Regression (l1) на валидационной выборке: 0.3036598141922943
AUC-ROC для Logistic Regression (l1) на валидационной выборке: 0.7660286271699316

F1_score для Logistic Regression (l2) на валидационной выборке: 0.30417552985749013
AUC-ROC для Logistic Regression (l2) на валидационной выборке: 0.7662704053193184


In [23]:
data_non_balance = [['decision_tree', 
                     sum(f1_score_val_dt)/len(f1_score_val_dt), sum(auc_roc_val_dt)/len(auc_roc_val_dt)], 
                    ['random_forest', 
                     sum(f1_score_val_rf)/len(f1_score_val_rf), sum(auc_roc_val_rf)/len(auc_roc_val_rf)], 
                    ['logistic_regression (l1)', 
                     sum(f1_score_val_lr_1)/len(f1_score_val_lr_1), sum(auc_roc_val_lr_1)/len(auc_roc_val_lr_1)],
                    ['logistic_regression (l2)', 
                     sum(f1_score_val_lr_2)/len(f1_score_val_lr_2), sum(auc_roc_val_lr_2)/len(auc_roc_val_lr_2)]]
columns_non_balance = ['model', 'f1_non_balance', 'auc_roc_non_balance']
table_non_balance = pd.DataFrame(data = data_non_balance, columns = columns_non_balance)
table_non_balance

,model,f1_non_balance,auc_roc_non_balance
0,decision_tree,0.509669,0.758082
1,random_forest,0.531694,0.829371
2,logistic_regression (l1),0.303660,0.766029
3,logistic_regression (l2),0.304176,0.766270


### Вывод

<ul> Для трех типов моделей были определены гиперпараметры, необходимые для достижения максимально возможной F1 меры. Как видно из таблицы, наибольшим значением обладает модель случайного леса. Близким значением обладает модель решающего дерева, а логистическая регрессия показала наихудший результат, равный 0.3. Вероятно, такие результаты связаны с тем, что мы не учитываем дисбаланс классов. Стоит отметить, что необходимого результата мы пока что не достигли (все значения F1 меньше 0.59). </ul>

# 3. Борьба с дисбалансом

Первая модель - решающее дерево.

In [24]:
dt_balanced = DecisionTreeClassifier(random_state = 12345, class_weight = 'balanced')
search_dt_balanced = RandomizedSearchCV(dt_balanced, parameters_dt, cv=5, n_jobs = -1, scoring = 'f1')
search_dt_balanced.fit(features_train, target_train)
best_dt_balanced = search_dt_balanced.best_estimator_
f1_score_val_dt_b = cross_val_score(best_dt_balanced, features_valid, target_valid, scoring = 'f1', cv = 10, n_jobs = -1)   
auc_roc_val_dt_b = cross_val_score(best_dt_balanced, features_valid, target_valid, scoring = 'roc_auc', cv = 10, n_jobs = -1)
print('F1-score для Decision Tree (balanced) на валидационной выборке:', sum(f1_score_val_dt_b)/len(f1_score_val_dt_b))
print('AUC-ROC для Decision Tree (balanced) на валидационной выборке:', sum(auc_roc_val_dt_b)/len(auc_roc_val_dt_b))

F1_score для Decision Tree (balanced) на валидационной выборке: 0.5017278460678608
AUC-ROC для Decision Tree (balanced) на валидационной выборке: 0.7649570492912885


Использование downsampling.

In [25]:
# Напишем функцию для реализации downsampling
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.255)
target_downsampled.value_counts()

1    1219
0    1219
Name: exited, dtype: int64

In [26]:
search_dt.fit(features_downsampled, target_downsampled)
best_dt_ds = search_dt.best_estimator_
f1_score_val_dt_ds = cross_val_score(best_dt_ds, features_valid, target_valid, scoring = 'f1', cv = 10, n_jobs = -1)   
auc_roc_val_dt_ds = cross_val_score(best_dt_ds, features_valid, target_valid, scoring = 'roc_auc', cv = 10, n_jobs = -1)
print('F1-score для Decision Tree (downsampled) на валидационной выборке:', sum(f1_score_val_dt_ds)/len(f1_score_val_dt_ds))
print('AUC-ROC для Decision Tree (downsampled) на валидационной выборке:', sum(auc_roc_val_dt_ds)/len(auc_roc_val_dt_ds))

F1_score для Decision Tree (downsampled) на валидационной выборке: 0.4616144692864653
AUC-ROC для Decision Tree (downsampled) на валидационной выборке: 0.7940283086478738


Используем upsampling.

In [27]:
# Напишем функцию для upsamling
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)
target_upsampled.value_counts()

1    4876
0    4781
Name: exited, dtype: int64

In [28]:
search_dt.fit(features_upsampled, target_upsampled)
best_dt_us = search_dt.best_estimator_
f1_score_val_dt_us = cross_val_score(best_dt_us, features_valid, target_valid, scoring = 'f1', cv = 10, n_jobs = -1)   
auc_roc_val_dt_us = cross_val_score(best_dt_us, features_valid, target_valid, scoring = 'roc_auc', cv = 10, n_jobs = -1)
print('F1-score для Decision Tree (upsampled) на валидационной выборке:', sum(f1_score_val_dt_us)/len(f1_score_val_dt_us))
print('AUC-ROC для Decision Tree (upsampled) на валидационной выборке:', sum(auc_roc_val_dt_us)/len(auc_roc_val_dt_us))

F1_score для Decision Tree (upsampled) на валидационной выборке: 0.4787685564467261
AUC-ROC для Decision Tree (upsampled) на валидационной выборке: 0.747300326485109


Вторая модель - случайный лес.

In [29]:
rf_balanced = RandomForestClassifier(random_state = 12345, class_weight = 'balanced')
search_rf_balanced = RandomizedSearchCV(rf_balanced, parameters_rf, cv=5, n_jobs = -1, scoring = 'f1')
search_rf_balanced.fit(features_train, target_train)
best_rf_balanced = search_rf_balanced.best_estimator_
f1_score_val_rf_b = cross_val_score(best_rf_balanced, features_valid, target_valid, scoring = 'f1', cv = 10, n_jobs = -1)   
auc_roc_val_rf_b = cross_val_score(best_rf_balanced, features_valid, target_valid, scoring = 'roc_auc', cv = 10, n_jobs = -1)
print('F1-score для Random Forest (balanced) на валидационной выборке:', sum(f1_score_val_rf_b)/len(f1_score_val_rf_b))
print('AUC-ROC для Random Forest (balanced) на валидационной выборке:', sum(auc_roc_val_rf_b)/len(auc_roc_val_rf_b))

F1_score для Random Forest (balanced) на валидационной выборке: 0.5702640033470395
AUC-ROC для Random Forest (balanced) на валидационной выборке: 0.8440260192705846


Методика downsampling для случайного леса

In [30]:
search_rf.fit(features_downsampled, target_downsampled)
best_rf_ds = search_rf.best_estimator_
f1_score_val_rf_ds = cross_val_score(best_rf_ds, features_valid, target_valid, scoring = 'f1', cv = 10, n_jobs = -1)   
auc_roc_val_rf_ds = cross_val_score(best_rf_ds, features_valid, target_valid, scoring = 'roc_auc', cv = 10, n_jobs = -1)
print('F1-score для Random Forest (downsampled) на валидационной выборке:', sum(f1_score_val_rf_ds)/len(f1_score_val_rf_ds))
print('AUC-ROC для Random Forest (downsampled) на валидационной выборке:', sum(auc_roc_val_rf_ds)/len(auc_roc_val_rf_ds))

F1_score для Random Forest (downsampled) на валидационной выборке: 0.5225081536156467
AUC-ROC для Random Forest (downsampled) на валидационной выборке: 0.8431139512661252


Методика upsampling для случайного леса

In [31]:
search_rf.fit(features_upsampled, target_upsampled)
best_rf_us = search_rf.best_estimator_
f1_score_val_rf_us = cross_val_score(best_rf_us, features_valid, target_valid, scoring = 'f1', cv = 10, n_jobs = -1)   
auc_roc_val_rf_us = cross_val_score(best_rf_us, features_valid, target_valid, scoring = 'roc_auc', cv = 10, n_jobs = -1)
print('F1-score для Random Forest (upsampled) на валидационной выборке:', sum(f1_score_val_rf_us)/len(f1_score_val_rf_us))
print('AUC-ROC для Random Forest (upsampled) на валидационной выборке:', sum(auc_roc_val_rf_us)/len(auc_roc_val_rf_us))

F1_score для Random Forest (upsampled) на валидационной выборке: 0.5275176986445127
AUC-ROC для Random Forest (upsampled) на валидационной выборке: 0.833615922121357


Третья модель - логистическая регрессия.

In [32]:
lr_balanced = LogisticRegression(random_state = 12345, class_weight = 'balanced')

search_lr_1_balanced = RandomizedSearchCV(lr_balanced, parameters_lr_1, cv=5, n_jobs = -1, scoring = 'f1')
search_lr_1_balanced.fit(features_train, target_train)
best_lr_1_b = search_lr_1_balanced.best_estimator_
f1_score_val_lr_1_b = cross_val_score(best_lr_1_b, features_valid, target_valid, scoring = 'f1', cv = 10, n_jobs = -1)   
auc_roc_val_lr_1_b = cross_val_score(best_lr_1_b, features_valid, target_valid, scoring = 'roc_auc', cv = 10, n_jobs = -1) 


search_lr_2_balanced = RandomizedSearchCV(lr_balanced, parameters_lr_2, cv=5, n_jobs = -1, scoring = 'f1')
search_lr_2_balanced.fit(features_train, target_train)
best_lr_2_b = search_lr_2_balanced.best_estimator_
f1_score_val_lr_2_b = cross_val_score(best_lr_2_b, features_valid, target_valid, scoring = 'f1', cv = 10, n_jobs = -1)   
auc_roc_val_lr_2_b = cross_val_score(best_lr_2_b, features_valid, target_valid, scoring = 'roc_auc', cv = 10, n_jobs = -1) 
print('F1-score для Logistic Regression (l1, balanced) на валидационной выборке:', 
      sum(f1_score_val_lr_1_b)/len(f1_score_val_lr_1_b))
print('AUC-ROC для Logistic Regression (l1, balanced) на валидационной выборке:', 
      sum(auc_roc_val_lr_1_b)/len(auc_roc_val_lr_1_b))
print()
print('F1-score для Logistic Regression (l2, balanced) на валидационной выборке:', 
      sum(f1_score_val_lr_2_b)/len(f1_score_val_lr_2_b))
print('AUC-ROC для Logistic Regression (l2, balanced) на валидационной выборке:', 
      sum(auc_roc_val_lr_2_b)/len(auc_roc_val_lr_2_b))

F1_score для Logistic Regression (l1, balanced) на валидационной выборке: 0.47856300591640555
AUC-ROC для Logistic Regression (l1, balanced) на валидационной выборке: 0.7695491917502788

F1_score для Logistic Regression (l2, balanced) на валидационной выборке: 0.4801724574049092
AUC-ROC для Logistic Regression (l2, balanced) на валидационной выборке: 0.7698713967192228


Методика downsampling для логистической регрессии

In [33]:
search_lr_1.fit(features_downsampled, target_downsampled)
best_lr_1_ds = search_lr_1.best_estimator_
f1_score_val_lr_1_ds = cross_val_score(best_lr_1_ds, features_valid, target_valid, scoring = 'f1', cv = 10, n_jobs = -1)   
auc_roc_val_lr_1_ds = cross_val_score(best_lr_1_ds, features_valid, target_valid, scoring = 'roc_auc', cv = 10, n_jobs = -1) 


search_lr_2.fit(features_downsampled, target_downsampled)
best_lr_2_ds = search_lr_2.best_estimator_
f1_score_val_lr_2_ds = cross_val_score(best_lr_2_ds, features_valid, target_valid, scoring = 'f1', cv = 10, n_jobs = -1)   
auc_roc_val_lr_2_ds = cross_val_score(best_lr_2_ds, features_valid, target_valid, scoring = 'roc_auc', cv = 10, n_jobs = -1) 

print('F1-score для Logistic Regression (l1, downsampled) на валидационной выборке:', 
      sum(f1_score_val_lr_1_ds)/len(f1_score_val_lr_1_ds))
print('AUC-ROC для Logistic Regression (l1, downsampled) на валидационной выборке:', 
      sum(auc_roc_val_lr_1_ds)/len(auc_roc_val_lr_1_ds))
print()
print('F1-score для Logistic Regression (l2, downsampled) на валидационной выборке:', 
      sum(f1_score_val_lr_2_ds)/len(f1_score_val_lr_2_ds))
print('AUC-ROC для Logistic Regression (l2, downsampled) на валидационной выборке:', 
      sum(auc_roc_val_lr_2_ds)/len(auc_roc_val_lr_2_ds))

F1_score для Logistic Regression (l1, downsampled) на валидационной выборке: 0.2787158537929946
AUC-ROC для Logistic Regression (l1, downsampled) на валидационной выборке: 0.7670606386367256

F1_score для Logistic Regression (l2, downsampled) на валидационной выборке: 0.26380855996516095
AUC-ROC для Logistic Regression (l2, downsampled) на валидационной выборке: 0.7685331063863673


Методика upsampling для логистической регрессии

In [34]:
search_lr_1.fit(features_upsampled, target_upsampled)
best_lr_1_us = search_lr_1.best_estimator_
f1_score_val_lr_1_us = cross_val_score(best_lr_1_us, features_valid, target_valid, scoring = 'f1', cv = 10, n_jobs = -1)   
auc_roc_val_lr_1_us = cross_val_score(best_lr_1_us, features_valid, target_valid, scoring = 'roc_auc', cv = 10, n_jobs = -1) 


search_lr_2.fit(features_upsampled, target_upsampled)
best_lr_2_us = search_lr_2.best_estimator_
f1_score_val_lr_2_us = cross_val_score(best_lr_2_us, features_valid, target_valid, scoring = 'f1', cv = 10, n_jobs = -1)   
auc_roc_val_lr_2_us = cross_val_score(best_lr_2_us, features_valid, target_valid, scoring = 'roc_auc', cv = 10, n_jobs = -1) 

print('F1-score для Logistic Regression (l1, upsampled) на валидационной выборке:', 
      sum(f1_score_val_lr_1_us)/len(f1_score_val_lr_1_us))
print('AUC-ROC для Logistic Regression (l1, upsampled) на валидационной выборке:', 
      sum(auc_roc_val_lr_1_us)/len(auc_roc_val_lr_1_us))
print()
print('F1-score для Logistic Regression (l2, upsampled) на валидационной выборке:', 
      sum(f1_score_val_lr_2_us)/len(f1_score_val_lr_2_us))
print('AUC-ROC для Logistic Regression (l2, upsampled) на валидационной выборке:', 
      sum(auc_roc_val_lr_2_us)/len(auc_roc_val_lr_2_us))

F1_score для Logistic Regression (l1, upsampled) на валидационной выборке: 0.22621713611226898
AUC-ROC для Logistic Regression (l1, upsampled) на валидационной выборке: 0.7678013019589107

F1_score для Logistic Regression (l2, upsampled) на валидационной выборке: 0.26380855996516095
AUC-ROC для Logistic Regression (l2, upsampled) на валидационной выборке: 0.7685330068482242


In [35]:
data_balance = [['decision_tree', sum(f1_score_val_dt)/len(f1_score_val_dt), 
               sum(f1_score_val_dt_b)/len(f1_score_val_dt_b), 
               sum(f1_score_val_dt_us)/len(f1_score_val_dt_us), 
               sum(f1_score_val_dt_ds)/len(f1_score_val_dt_ds)], 
              ['random_forest', sum(f1_score_val_rf)/len(f1_score_val_rf), 
               sum(f1_score_val_rf_b)/len(f1_score_val_rf_b), 
               sum(f1_score_val_rf_us)/len(f1_score_val_rf_us), 
               sum(f1_score_val_rf_ds)/len(f1_score_val_rf_ds)], 
              ['logistic_regression (l1)', sum(f1_score_val_lr_1)/len(f1_score_val_lr_1), 
               sum(f1_score_val_lr_1_b)/len(f1_score_val_lr_1_b), 
               sum(f1_score_val_lr_1_us)/len(f1_score_val_lr_1_us), 
               sum(f1_score_val_lr_1_ds)/len(f1_score_val_lr_1_ds)],
              ['logistic_regression (l2)', sum(f1_score_val_lr_2)/len(f1_score_val_lr_2), 
               sum(f1_score_val_lr_2_b)/len(f1_score_val_lr_2_b), 
               sum(f1_score_val_lr_2_us)/len(f1_score_val_lr_2_us), 
               sum(f1_score_val_lr_2_ds)/len(f1_score_val_lr_2_ds)]]
columns_balance = ['model', 'f1_non_balance', 'f1_balance', 'f1_upsample', 'f1_downsample']
table_balance = pd.DataFrame(data = data_balance, columns = columns_balance)
table_balance

,model,f1_non_balance,f1_balance,f1_upsample,f1_downsample
0,decision_tree,0.509669,0.501728,0.478769,0.461614
1,random_forest,0.531694,0.570264,0.527518,0.522508
2,logistic_regression (l1),0.303660,0.478563,0.226217,0.278716
3,logistic_regression (l2),0.304176,0.480172,0.263809,0.263809


### Вывод

<ul> Как видно из таблицы выше, учитывая дисбаланс класса, мы смогли достичь большего значения F1-меры. </ul>
   <ul> Для модели решающего дерева эффективнее всего было учитывывать дисбаланс через настройку модели, а методики upsampling и downsampling лишь уменьшили F1-меру. </ul>
   <ul> Для случайного леса наилучший результат показала модель с учетом дисбаланса через настройку модели и методики downsample. </ul>
   <ul> Для логистической регрессии учет дисбаланса привел к увеличению F1-меры.</ul>

# 4. Тестирование модели

In [36]:
# Так как наилучший результат показала модель случайного леса с балансом классов, настроенном через параметры модели,
# протестируем именно её
f1_score_test = cross_val_score(best_rf_balanced, features_test, target_test, scoring = 'f1', cv = 10, n_jobs = -1)   
auc_roc_test = cross_val_score(best_rf_balanced, features_test, target_test, scoring = 'roc_auc', cv = 10, n_jobs = -1)
print('F1-score для Random Forest (balanced) на тестовой выборке:', sum(f1_score_test)/len(f1_score_test))
print('AUC-ROC для Random Forest (balanced) на тестовой выборке:', sum(auc_roc_test)/len(auc_roc_test))

F1_score для Random Forest (balanced) на тестовой выборке: 0.630686148753337
AUC-ROC для Random Forest (balanced) на тестовой выборке: 0.8649864744436762


### Вывод

<ul> В ходе работы были найдена наилучшая модель, которая на тестовых данных показала значения метрики F1 больше, чем 0.59. Таким образом, необходимая задача была выполнена. </ul>